### 분석 목적: 유튜브 조회 수, 좋아요, 싫어요 수를 통한 오디션 프로그램의 우승자 예측

### 분석 배경
- 2021년 12월 6일날 첫 방영한 "싱어게인2" 프로그램에서 우승자를 예측하기 위함
- 유튜브 조회 수는 '화제성'을 상징하는 지표라고 생각하고, '좋아요'와 '싫어요'는 대중들의 평가를 상징하는 지표라고 생각
- 이 지표들을 분석함으로써 실제 오디션 프로그램의 우승자를 예측해보려고 함



[url](https://www.youtube.com/watch?v=D0HfiSWP6-Y) 참고해서 적어보자


In [1]:
import numpy as np
import MySQLdb
import pandas as pd
import re
import requests
import time

from datetime import datetime

In [2]:
# connect mysql
conn = MySQLdb.connect(host="*******", 
                       port=3306, 
                       user='root', 
                       password='*******',
                       database='singAgain',
                       charset='utf8')
cursor = conn.cursor()

# Select columns
sql = "Show columns from youtubeInfo"
cursor.execute(sql)
columns = []
for col in cursor.fetchall():
    columns.append(col[0])

# Select all data
sql = "Select * from youtubeInfo"
cursor.execute(sql)
values = cursor.fetchall()

In [3]:
# 영상 올린 날짜 추가 
#my_id = '*******'
#video_url = f'https://www.googleapis.com/youtube/v3/videos?part=id,snippet,statistics&id=bzM82b6QrnA&key={my_id}'
#result = requests.get(video_url)
#result_dict = result.json()
#result_dict['items'][0]['snippet']['publishedAt']

In [4]:
sA_aws_df = pd.DataFrame(values, columns=columns)

In [5]:
# 기존 데이터 얻은 것과 결합
# connect mysql
conn = MySQLdb.connect(host="*******", 
                       port=3306, 
                       user='root', 
                       password='*******',
                       database='singAgain',
                       charset='utf8')
cursor = conn.cursor()

# Select columns
sql = "Show columns from youtubeInfo"
cursor.execute(sql)
columns = []
for col in cursor.fetchall():
    columns.append(col[0])

# Select all data
sql = "Select * from youtubeInfo"
cursor.execute(sql)
values = cursor.fetchall()

In [6]:
sA_df = pd.DataFrame(values, columns=columns)
sA_df = pd.concat([sA_df, sA_aws_df], axis=0)
sA_df = sA_df.reset_index(drop=True)

## 데이터 정제

In [7]:
# 참가자 번호 get
def get_participant_no(string):
    par_no = re.findall("[1-99]+[호]", string)
    if len(par_no) > 0:
        par_no = par_no[0]
    else:
        par_no = np.nan
    return par_no
sA_df['par_no'] = sA_df['title'].apply(get_participant_no)

In [8]:
# keyword 생성
sA_df['keyword'] = np.nan
sA_df.loc[sA_df['title'].str.contains("1열 직캠") == True, 'keyword'] = "1열 직캠"
sA_df.loc[sA_df['title'].str.contains("핫클립") == True, 'keyword'] = "핫클립"
sA_df.loc[sA_df['title'].str.contains("끌립") == True, 'keyword'] = "전체끌립"
sA_df.loc[sA_df['title'].str.contains("퀵클립") == True, 'keyword'] = "퀵클립"
sA_df.loc[sA_df['title'].str.contains("선공개") == True, 'keyword'] = "선공개"
sA_df.loc[sA_df['title'].str.contains("예고") == True, 'keyword'] = "예고"
sA_df.loc[sA_df['title'].str.contains("모아듣기") == True, 'keyword'] = "모아듣기"
sA_df.loc[(sA_df['par_no'].isnull() == False) & (sA_df['keyword'].isnull()), 'keyword'] = "무대"

In [9]:
# keyword 없는 행 삭제
drop_index = sA_df[sA_df['keyword'].isnull()].index
sA_df = sA_df.drop(index=drop_index)
sA_df = sA_df.reset_index(drop=True)

# title 삭제 
sA_df = sA_df.drop(columns='title')

In [10]:
sA_df

,time,viewCount,likeCount,favoriteCount,commentCount,par_no,keyword
0,2021-12-14 03:08:27,119403,1429,0,168,24호,1열 직캠
1,2021-12-14 03:08:28,382036,5616,0,482,31호,1열 직캠
2,2021-12-14 03:08:28,88561,1145,0,287,63호,1열 직캠
3,2021-12-14 03:08:28,67664,732,0,74,27호,1열 직캠
4,2021-12-14 03:08:28,76290,1097,0,187,42호,1열 직캠
...,...,...,...,...,...,...,...
5236,2021-12-18 19:30:12,459422,2685,0,227,2호,선공개
5237,2021-12-18 19:30:12,278834,5458,0,363,64호,무대
5238,2021-12-18 19:30:12,97526,642,0,59,NaN,모아듣기
5239,2021-12-18 19:30:12,33173,276,0,18,NaN,모아듣기
